In [1]:
from utils.experimentor_base import *
from utils.experimentor_proteins import * 
from utils.constants import *
print(torch.backends.cudnn.version())

SyntaxError: invalid syntax (Temp/ipykernel_7128/2628189836.py, line 3)

In [ ]:
config = {
    "dataset_name": Dataset.OGBN_PRODUCTS,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 500,
    'num_of_runs': 1,
    'patience_period': 100,
    
    'batch_size': 256,
    'test_batch_size': 256,
    'num_workers': 2,
    'force_cpu': False,
    'test_frequency': 1,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    
    'lr': 0.001,
    'num_of_layers': 3, 
    'num_heads': 1,
    'hidden_size': 128,
    'dropout': 0.5,  
    "use_layer_norm": False,
    "use_batch_norm": False,
    
    'nbor_degree': 2,
    'adj_mode': AdjacencyMode.OneStep,
    'sparse': True
}

In [ ]:
exp = Experimentor_Proteins if config["dataset_name"] == Dataset.OGBN_PROTEINS else Experimentor
experimentor = exp(config)
experimentor.run_wrapper()